# Titanic Case Study 
Bintu Labibah 

Dataset
https://drive.google.com/file/d/1v7s3hoK6RKjtNdvzMs9KQKaUr7Nc41XX/view

In [767]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [768]:
# Load CSV data
data = pd.read_csv('titanic-train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Data Checking

In [769]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [770]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [771]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Note: Terdapat beberapa kolom yang bersifat non numerik, selanjutnya kita akan handle dengan DROP kolom dan ENCODE kolom yang bisa dijadikan numerik serta mengisi missing value dengan mean dan mode.

In [772]:
# Drop kolom Name, Ticket, Cabin
data = data.drop(columns=['Name', 'Ticket', 'Cabin'])

# Kolom ini dihapus karena tidak berpengaruh terhadap model yang akan dibuat karena values dari kolom-kolom tersebut hanya berisi nama, 
# nomor tiket dan nomor kabin dimana target kita adalah untuk mengetahui berapa orang yang selamat dan yang meninggal. Selain itu, 
# kolom ini memiliki type data object sehingga untuk 3 kolom ini tidak cukup mudah untuk dilakukan encode

In [773]:
# Encoding kolom Sex (Male = 1, Female = 0)
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])

# Kolom Sex masih bisa dilakukan encode dimana 1 untuk male dan 0 untuk female, 
# kolom ini juga akan berpengaruh terhadap tingkat survived

In [774]:
# Mengisi missing value di kolom Age dengan rata-rata usia
mean_age = data['Age'].mean()

# Isi nilai NaN dalam kolom Age dengan mean
data['Age'] = data['Age'].fillna(mean_age)

# Kolom Age memiliki nilai null sebanyak 177 maka dari itu kita mengisinya dengan 
# menggunakan mean dari kolom Age karena type data pada kolom tersebut adalah numerik

In [775]:
# Mengisi missing value di kolom Embarked dengan nilai modus
modus = data['Embarked'].mode().iloc[0]

# Fill NaN value di kolom 'Embarked' dengan nilai modus
data['Embarked'] = data['Embarked'].fillna(modus)


# Kolom Embarked memiliki nilai null sebanyak 2 maka dari itu kita mengisinya dengan 
# menggunakan modus dari kolom Embarked karena type data pada kolom tersebut adalah non numerik

In [776]:
# Encoding kolom Embarked (C = 0, Q = 1, S = 2) 
le = LabelEncoder()
data['Embarked'] = le.fit_transform(data['Embarked'])
data

# Kolom Embarked masih bisa dilakukan encode dimana C = 0, Q = 1, S = 2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.000000,1,0,7.2500,2
1,2,1,1,0,38.000000,1,0,71.2833,0
2,3,1,3,0,26.000000,0,0,7.9250,2
3,4,1,1,0,35.000000,1,0,53.1000,2
4,5,0,3,1,35.000000,0,0,8.0500,2
...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,13.0000,2
887,888,1,1,0,19.000000,0,0,30.0000,2
888,889,0,3,0,29.699118,1,2,23.4500,2
889,890,1,1,1,26.000000,0,0,30.0000,0


In [777]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

## Split data

In [778]:
# Asumsikan nama kolom target dengan 'target'
X = data.drop('Survived', axis=1)
y = data['Survived']

In [779]:
# Encode categorical variables if necessary
# If your dataset contains categorical variables, you may need to encode them.
# For simplicity, let's assume all features are numeric.

# Split dataset into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Disini kita melakukan split data 70% untuk data train dan 20% untuk data test

In [780]:
X_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,332,1,1,45.500000,0,0,28.5000,2
733,734,2,1,23.000000,0,0,13.0000,2
382,383,3,1,32.000000,0,0,7.9250,2
704,705,3,1,26.000000,1,0,7.8542,2
813,814,3,0,6.000000,4,2,31.2750,2
...,...,...,...,...,...,...,...,...
106,107,3,0,21.000000,0,0,7.6500,2
270,271,1,1,29.699118,0,0,31.0000,2
860,861,3,1,41.000000,2,0,14.1083,2
435,436,1,0,14.000000,1,2,120.0000,2


In [781]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

In [782]:
X_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,710,3,1,29.699118,1,1,15.2458,0
439,440,2,1,31.000000,0,0,10.5000,2
840,841,3,1,20.000000,0,0,7.9250,2
720,721,2,0,6.000000,0,1,33.0000,2
39,40,3,0,14.000000,1,0,11.2417,0
...,...,...,...,...,...,...,...,...
433,434,3,1,17.000000,0,0,7.1250,2
773,774,3,1,29.699118,0,0,7.2250,0
25,26,3,0,38.000000,1,5,31.3875,2
84,85,2,0,17.000000,0,0,10.5000,2


## Membuat dan melatih model

In [784]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

# Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

In [785]:
print ('knn_pred: ', knn_pred),
print ('dt_pred: ', dt_pred), 
print ('rf_pred: ', rf_pred), 
print ('xgb_pred: ', xgb_pred)

knn_pred:  [0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0
 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0]
dt_pred:  [0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1
 1 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1
 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1]
rf_pred:  [0 0 0 1 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1
 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1

## Confusion Matrix

In [786]:
actual = y_test  # Actual target values
# Create confusion matrices
knn_cm = confusion_matrix(actual, knn_pred)
dt_cm = confusion_matrix(actual, dt_pred)
rf_cm = confusion_matrix(actual, rf_pred)
xgb_cm = confusion_matrix(actual, xgb_pred)

# Convert confusion matrices to DataFrames
def confusion_matrix_to_dataframe(cm):
    labels = sorted(set(actual))
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    return df_cm

# Convert confusion matrices to DataFrames with labels
knn_cm_df = confusion_matrix_to_dataframe(knn_cm)
dt_cm_df = confusion_matrix_to_dataframe(dt_cm)
rf_cm_df = confusion_matrix_to_dataframe(rf_cm)
xgb_cm_df = confusion_matrix_to_dataframe(xgb_cm)

In [787]:
knn_cm_df

Predicted,0,1
Actual,,
0,89,16
1,45,29


In [788]:
dt_cm_df

Predicted,0,1
Actual,,
0,83,22
1,23,51


In [789]:
rf_cm_df

Predicted,0,1
Actual,,
0,93,12
1,19,55


In [791]:
xgb_cm_df

Predicted,0,1
Actual,,
0,88,17
1,20,54


## Evaluate model

In [792]:
def evaluate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return cm, accuracy, precision, recall, f1


# Evaluate KNN
knn = knn_cm, knn_accuracy, knn_precision, knn_recall, knn_f1 = evaluate(y_test, knn_pred)

# Evaluate Decision Tree
dt = dt_cm, dt_accuracy, dt_precision, dt_recall, dt_f1 = evaluate(y_test, dt_pred)

# Evaluate Random Forest
rf = rf_cm, rf_accuracy, rf_precision, rf_recall, rf_f1 = evaluate(y_test, rf_pred)

# Evaluate XGBoost
xgb = xgb_cm, xgb_accuracy, xgb_precision, xgb_recall, xgb_f1 = evaluate(y_test, xgb_pred)

In [793]:
print ('knn confusion matrix', knn_cm)
print ('knn accuracy', knn_accuracy)
print ('knn precission', knn_precision)
print ('knn recall', knn_recall)
print ('knn F1 score', knn_f1)

knn confusion matrix [[89 16]
 [45 29]]
knn accuracy 0.659217877094972
knn precission 0.6560206416706043
knn recall 0.659217877094972
knn F1 score 0.6383690710102792


In [794]:
print ('dt confusion matrix', dt_cm)
print ('dt accuracy', dt_accuracy)
print ('dt precission', dt_precision)
print ('dt recall', dt_recall)
print ('dt F1 score', dt_f1)

dt confusion matrix [[83 22]
 [23 51]]
dt accuracy 0.7486033519553073
dt precission 0.7481319065310713
dt recall 0.7486033519553073
dt F1 score 0.7483439879698338


In [795]:
print ('rf confusion matrix', rf_cm)
print ('rf accuracy', rf_accuracy)
print ('rf precission', rf_precision)
print ('rf recall', rf_recall)
print ('rf F1 score', rf_f1)

rf confusion matrix [[93 12]
 [19 55]]
rf accuracy 0.8268156424581006
rf precission 0.8264456349537231
rf recall 0.8268156424581006
rf F1 score 0.8253100360553113


In [796]:
print ('xgb confusion matrix', xgb_cm)
print ('xgb accuracy', xgb_accuracy)
print ('xgb precission', xgb_precision)
print ('xgb recall', xgb_recall)
print ('xgb F1 score', xgb_f1)

xgb confusion matrix [[88 17]
 [20 54]]
xgb accuracy 0.7932960893854749
xgb precission 0.7923868474659252
xgb recall 0.7932960893854749
xgb F1 score 0.7926134344111287


### Note: Dari keempat model tersebut, model yang memiliki tingkat akurasi yang paling tinggi adalah 'Random Forest' dengan menunjukkan kemampuan terbaik dalam memprediksi penumpang yang selamat (82.68%). Selain itu,  RF menunjukkan keseimbangan antara kemampuan memprediksi penumpang yang selamat (presisi) dan yang tidak selamat (recall). Dimana recall memprediksi lebih banyak penumpang selamat karena recall yang tinggi, dan dapat membantu menurunkan false negatif sehingga ketika kita memprediksi lebih banyak penumpang selamat dimana dalam kasus ini kita bisa melakukan pencarian dan penyelamatan lebih cepat walaupun mungkin pada kenyataannya penumpang tersebut telah meninggal. Akan tetapi sisi positifnya kita bisa menemukan jasadnya, dan tidak meninggikan resiko karena yang di prediksi meninggal lebih kecil, karena takutnya pada kenyataannya dia masih hidup tetapi di presiksi telah meninggal, itu bisa saja pencarian/penyelamatannya dikesampingkan. serta di F1 Score yang menggabungkan presisi dan recall memiliki nilai tinggi yang menunjukkan performa model yang optimal dalam klasifikasi. 

## Feature importance

### XGB model feature importance

In [828]:
xgb_feature_importance = xgb_model.feature_importances_
feature_names = X.columns

xgb_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': xgb_feature_importance})
xgb_feature_importance_df = xgb_feature_importance_df.sort_values(by='Importance', ascending=False)
xgb_feature_importance_df

,Feature,Importance
2,Sex,0.550486
1,Pclass,0.186398
4,SibSp,0.077695
3,Age,0.040443
6,Fare,0.039936
5,Parch,0.037542
7,Embarked,0.034123
0,PassengerId,0.033376


Kolom dengan Importance Tinggi:

Sex (Jenis Kelamin): Memiliki importance tertinggi (0.550486) menunjukkan bahwa jenis kelamin merupakan faktor paling penting dalam menentukan apakah seseorang akan selamat atau tidak. Wanita memiliki peluang lebih tinggi untuk selamat dibandingkan pria. Hal ini mungkin disebabkan oleh beberapa faktor, seperti:
Wanita secara fisik lebih kuat dan memiliki tingkat lemak tubuh yang lebih tinggi, yang dapat membantu mereka bertahan dalam kondisi ekstrem.
Wanita lebih cenderung didahulukan dalam evakua

si.
Pclass (Kelas Tiket): Memiliki importance kedua tertinggi (0.186398) menunjukkan bahwa kelas tiket juga merupakan faktor penting. Penumpang kelas 1 (kelas atas) memiliki peluang lebih tinggi untuk selamat dibandingkan penumpang kelas 2 dan 3. Hal ini mungkin disebabkan oleh:
Fasilitas dan akses ke layanan penyelamatan yang lebih baik di kelas 1.
Kemungkinan lebih tinggi untuk didahulukan dalam eva
kuasi.
Kolom dengan Importance Rendah:

SibSp (Jumlah Saudara Kandung/Pasangan di Kapal): Memiliki importance yang relatif rendah (0.077695) menunjukkan bahwa jumlah saudara kandung atau pasangan di kapal tidak memiliki pengaruh yang besar pada peluang seseo ng untuk

 selamat.
Age (Usia): Memiliki importance yang relatif rendah (0.040443) menunjukkan bahwa usia juga tidak memiliki pengaruh yang besar pada peluang sesorang untu

k selamat.
Fare (Harga Tiket): Memiliki importance yang relatif rendah (0.039936) menunjukkan bahwa harga tiket tidak memiliki pengaruh yang besar pada peluang seseorang unt

uk selamat.
Parch (Jumlah Orang Tua/Anak di Kapal): Memiliki importance yang relatif rendah (0.037542) menunjukkan bahwa jumlah orang tua atau anak di kapal tidak memiliki pengaruh yang besar pada peluangseseorang un

tuk selamat.
Embarked (Pelabuhan Keberangkatan): Memiliki importance yang relatif rendah (0.034123) menunjukkan bahwa pelabuhan keberangkatan tidak memiliki pengaruh yang besar pada peluag seseorang u

ntuk selamat.
PassengerId (ID Penumpang): Memiliki importance yang relatif rendah (0.033376) menunjukkan bahwa ID penumpang tidak memiliki pengaruh pada peluang seseorang untuk selamat.ntuk selamat.

### RF model feature importance

In [825]:
rf_feature_importance = rf.feature_importances_
feature_names = X.columns

rf_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': rf_feature_importance})
rf_feature_importance_df = rf_feature_importance_df.sort_values(by='Importance', ascending=False)
rf_feature_importance_df

,Feature,Importance
2,Sex,0.242632
6,Fare,0.213040
0,PassengerId,0.194194
3,Age,0.170379
1,Pclass,0.074542
4,SibSp,0.043989
5,Parch,0.031957
7,Embarked,0.029267


Kolom dengan Importance Tinggi:

Sex (Jenis Kelamin): Memiliki importance tertinggi (0.242632) menunjukkan bahwa jenis kelamin merupakan faktor paling penting dalam menentukan apakah seseorang akan selamat atau tidak. Wanita memiliki peluang lebih tinggi untuk selamat dibandingkan pria. Hal ini mungkin disebabkan oleh beberapa faktor, seperti:
Wanita secara fisik lebih kuat dan memiliki tingkat lemak tubuh yang lebih tinggi, yang dapat membantu mereka bertahan dalam kondisi ekstrem.
Wanita lebih cenderung didahulukan dalam evakuasi.

Fare (Harga Tiket): Memiliki importance kedua tertinggi (0.213040) menunjukkan bahwa harga tiket juga merupakan faktor penting. Orang yang mampu membeli tiket kelas atas memiliki peluang lebih tinggi untuk selamat. Hal ini mungkin disebabkan oleh:
Fasilitas dan akses ke layanan penyelamatan yang lebih baik di kelas atas.
Kemungkinan lebih tinggi untuk didahulukan dalam evakuasi.

PassengerId (ID Penumpang): Memiliki importance ketiga tertinggi (0.194194). Perlu dicatat bahwa PassengerId adalah fitur unik yang tidak memiliki makna intrinsik dalam menentukan apakah seseorang akan selamat atau tidak. Kemungkinan importance tinggi PassengerId disebabkan oleh korelasi dengan fitur lain, seperti Pclass dan Fare.

Kolom dengan Importance Rendah:

Age (Usia): Memiliki importance yang relatif rendah (0.170379) menunjukkan bahwa usia tidak memiliki pengaruh yang besar pada peluang seseorang untuk selamat.

Pclass (Kelas Tiket): Memiliki importance yang relatif rendah (0.074542) dibandingkan dengan Sex dan Fare. Hal ini mungkin disebabkan oleh korelasi antara Pclass dengan Fare.

SibSp (Jumlah Saudara Kandung/Pasangan di Kapal): Memiliki importance yang relatif rendah (0.043989) menunjukkan bahwa jumlah saudara kandung atau pasangan di kapal tidak memiliki pengaruh yang besar pada peluang seseorang untuk selamat.

Parch (Jumlah Orang Tua/Anak di Kapal): Memiliki importance yang relatif rendah (0.031957) menunjukkan bahwa jumlah orang tua atau anak di kapal tidak memiliki pengaruh yang besar pada peluang seseorang untuk selamat.

Embarked (Pelabuhan Keberangkatan): Memiliki importance yang relatif rendah (0.029267) menunjukkan bahwa pelabuhan keberangkatan tidak memiliki pengaruh yang besar pada peluang seseorang untuk selamat.

### Note: Menurut saya feature importance dengan model XGB lebih relevan dan masuk di logika karena di RF setiap fitur walaupun berada diurutan pertama akan tetapi nilainya kecil. Jadi saya menampilkan keduanya untuk bisa dibandingkan.

## Prediksi model menggunakan data test

In [798]:
# Load new data
new_data = pd.read_csv('titanic-test.csv')
new_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [799]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [800]:
new_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [801]:
# Drop kolom Name, Ticket, Cabin
new_data = new_data.drop(columns=['Name', 'Ticket', 'Cabin'])

In [802]:
# Encoding kolom Sex (Male = 1, Female = 0)
le = LabelEncoder()
new_data['Sex'] = le.fit_transform(new_data['Sex'])

In [803]:
# Mengisi missing value di kolom Age dengan rata-rata usia
mean_age = new_data['Age'].mean()

# Isi nilai NaN dalam kolom Age dengan mean
new_data['Age'] = new_data['Age'].fillna(mean_age)

In [804]:
# Mengisi missing value di kolom Fare dengan rata-rata ongkos tiket
mean_fare = new_data['Fare'].mean()

# Isi nilai NaN dalam kolom Fare dengan mean
new_data['Fare'] = new_data['Fare'].fillna(mean_fare)

In [805]:
# Encoding kolom Embarked (C = 0, Q = 1, S = 2) 
le = LabelEncoder()
new_data['Embarked'] = le.fit_transform(new_data['Embarked'])
new_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.50000,0,0,7.8292,1
1,893,3,0,47.00000,1,0,7.0000,2
2,894,2,1,62.00000,0,0,9.6875,1
3,895,3,1,27.00000,0,0,8.6625,2
4,896,3,0,22.00000,1,1,12.2875,2
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,2
414,1306,1,0,39.00000,0,0,108.9000,0
415,1307,3,1,38.50000,0,0,7.2500,2
416,1308,3,1,30.27259,0,0,8.0500,2


In [806]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int32  
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    int32  
dtypes: float64(2), int32(2), int64(4)
memory usage: 23.0 KB


In [820]:
# Assuming your new data has the same features as your original data (except the target)

# Make predictions using the trained models
knn_predictions = knn.predict(new_data)
dt_predictions = dt.predict(new_data)
rf_predictions = rf.predict(new_data)
xgb_predictions = xgb_model.predict(new_data)

In [823]:
rf_predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [809]:
combined_data = new_data.copy()  # Copy the new dataset
combined_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.50000,0,0,7.8292,1
1,893,3,0,47.00000,1,0,7.0000,2
2,894,2,1,62.00000,0,0,9.6875,1
3,895,3,1,27.00000,0,0,8.6625,2
4,896,3,0,22.00000,1,1,12.2875,2
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,2
414,1306,1,0,39.00000,0,0,108.9000,0
415,1307,3,1,38.50000,0,0,7.2500,2
416,1308,3,1,30.27259,0,0,8.0500,2


## Hasil prediksi masing-masing model dan totalnya (Survuved 1 dan 0)

In [810]:
# Add columns for predicted labels
combined_data['KNN_Predictions'] = knn_predictions
combined_data['DT_Predictions'] = dt_predictions
combined_data['RF_Predictions'] = rf_predictions
combined_data['XGB_Predictions'] = xgb_predictions

In [811]:
combined_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,KNN_Predictions,DT_Predictions,RF_Predictions,XGB_Predictions
0,892,3,1,34.50000,0,0,7.8292,1,0,0,0,0
1,893,3,0,47.00000,1,0,7.0000,2,0,1,0,0
2,894,2,1,62.00000,0,0,9.6875,1,0,0,0,0
3,895,3,1,27.00000,0,0,8.6625,2,0,0,0,0
4,896,3,0,22.00000,1,1,12.2875,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,2,0,0,0,0
414,1306,1,0,39.00000,0,0,108.9000,0,1,1,1,1
415,1307,3,1,38.50000,0,0,7.2500,2,0,0,0,0
416,1308,3,1,30.27259,0,0,8.0500,2,0,0,0,0


In [812]:
# Total survived 1 dan 0 di KNN Predictions
knn_predictions_count_1 = (knn_predictions == 1).sum()
knn_predictions_count_0 = (knn_predictions == 0).sum()

print ('KNN Predictions 1: ', knn_predictions_count_1)
print ('KNN Predictions 0: ', knn_predictions_count_0)

KNN Predictions 1:  54
KNN Predictions 0:  364


In [813]:
# Total survived 1 dan 0 di DT Predictions
dt_predictions_count_1 = (dt_predictions == 1).sum()
dt_predictions_count_0 = (dt_predictions == 0).sum()

print ('DT Predictions 1: ', dt_predictions_count_1)
print ('DT Predictions 0: ', dt_predictions_count_0)

DT Predictions 1:  137
DT Predictions 0:  281


In [814]:
# Total survived 1 dan 0 di RF Predictions
rf_predictions_count_1 = (rf_predictions == 1).sum()
rf_predictions_count_0 = (rf_predictions == 0).sum()

print ('RF Predictions 1: ', rf_predictions_count_1)
print ('RF Predictions 0: ', rf_predictions_count_0)

RF Predictions 1:  147
RF Predictions 0:  271


In [815]:
# Total survived 1 dan 0 di XGB Predictions
xgb_predictions_count_1 = (xgb_predictions == 1).sum()
xgb_predictions_count_0 = (xgb_predictions == 0).sum()

print ('XGB Predictions 1: ', xgb_predictions_count_1)
print ('XGB Predictions 0: ', xgb_predictions_count_0)

XGB Predictions 1:  129
XGB Predictions 0:  289


### Note: Dari keempat model tersebut, model Random forest lebih banyak memprediksi penunmpang selamat (147) dan memprediksi lebih sedikit penumpang meninggal (271). Hal tersebut sesuai dengan model random forest yang memiliki tingkat akurasi dan recall diangka 82% dimana memprediksi lebih banyak penumpang selamat karena recall yang tinggi, dan dapat membantu menurunkan false negatif sehingga ketika kita memprediksi lebih banyak penumpang selamat dimana dalam kasus ini kita bisa melakukan pencarian dan penyelamatan lebih cepat walaupun mungkin pada kenyataannya penumpang tersebut telah meninggal. Akan tetapi sisi positifnya kita bisa menemukan jasadnya, dan tidak meninggikan resiko karena yang di prediksi meninggal lebih kecil, karena takutnya pada kenyataannya dia masih hidup tetapi di presiksi telah meninggal, itu bisa saja pencarian/penyelamatannya dikesampingkan.